# Тестирование гипотез

H₀ (Нулевая гипотеза): Доля женщин (в процентах) в университетах не зависит от их размера. Средние доли (или медианы, для непараметрического теста) генеральной совокупности равны во всех категориях размера.

H₁ (Альтернативная гипотеза): Доля женщин (в процентах) в университетах зависит от их размера. Существует хотя бы одна пара категорий размера, между которыми средние доли (медианы) генеральной совокупности статистически значимо различаются.

H₀ (Нулевая гипотеза): Доля вузов, где Research Quality заметно превышает Teaching, равна доле вузов, где Teaching заметно превышает Research Quality 

H₁ (Альтернативная гипотеза) - Исследовательская гипотеза: Доля вузов, где Research Quality заметно превышает Teaching, больше, чем доля вузов с обратной ситуацией.

H₀ (Нулевая гипотеза): Средняя доля элитных вузов в прикладных и междисциплинарных областях (Business and Economics, Medical and Health, Computer Science) одинакова в группе стран с высоким числом «универсалов» и в группе остальных стран.

H₁ (Альтернативная гипотеза) - Исследовательская гипотеза: Средняя доля элитных вузов в прикладных и междисциплинарных областях выше в группе стран с высоким числом «универсалов», чем в группе остальных стран.